# Requirements

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
import numpy as np
import scipy as sp
import scipy.integrate
import scipy.optimize
import sympy
output_notebook()

# No drag

The equations of motion of a projectile fired by a cannon is given by:
$$
  \frac{d^2 \vec{r}}{d t^2} = -g \vec{e}_y
$$
The value for $g$ is $9.81 m/s^2$, and the initial velocity $v_0$ is $700 m/s$.

In [ ]:
g = 9.81

## Equations

As usual, the second order differential equation is transformed into a set of first order equations, and the right hand side of the equations is defined by the function `rhs`.  Additionally, the Jacobian is provided.

In [ ]:
def rhs(t, Y, g):
    x, y, v_x, v_y = Y
    return np.array([
        v_x,
        v_y,
        0.0,
        -g,
    ])

In [ ]:
def jac(t, Y, g):
    x, y, v_x, v_y = Y
    v = np.sqrt(v_x**2 + v_y**2)
    return np.array([
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0],
    ])


Since it is more convenient to approach the problem using degrees rather than radians, a function is defined to compute the initial values.

In [ ]:
def compute_init_values(alpha, v_0 = 700):
    alpha_rad = np.pi*alpha/180.0
    return np.array([0.0, 0.0,
                     v_0*np.cos(alpha_rad), v_0*np.sin(alpha_rad),
                     ])

## Solution

We can solve the set of equations for an angle of 40 degrees.

In [ ]:
init_values = compute_init_values(40.0)
x_0, y_0, v_x_0, v_y_0 = init_values

Define the ODE system, setting initial conditions and parameters.

In [ ]:
sys = scipy.integrate.ode(rhs, jac) \
           .set_integrator('dopri5') \
           .set_initial_value(init_values, 0.0) \
           .set_f_params(g) \
           .set_jac_params(g)

In [ ]:
delta_t = 0.5e-2

In [ ]:
t, x, y, v_x, v_y = [0.0], [x_0], [y_0], [v_x_0], [v_y_0]

In [ ]:
while sys.successful() and y[-1] >= 0:
    sys.integrate(sys.t + delta_t)
    t.append(sys.t)
    x.append(sys.y[0])
    y.append(sys.y[1])
    v_x.append(sys.y[2])
    v_y.append(sys.y[3])

In [ ]:
fig = figure(width=500, height=300,
             x_axis_label='x', y_axis_label='y')
fig.line(x, y)
show(fig)

## Range as a function of the firing angle

In [ ]:
def compute_range(alpha, delta_t = 1.0e-3,
                  rhs=rhs, jac=jac,
                  rhs_params=(g, ), jac_params=(g, )):
    init_values = compute_init_values(alpha)
    x_0, y_0, v_x_0, v_y_0 = init_values
    sys = scipy.integrate.ode(rhs, jac) \
               .set_integrator('dopri5') \
               .set_initial_value(init_values, 0.0) \
               .set_f_params(*rhs_params) \
               .set_jac_params(*jac_params)
    t, x, y, v_x, v_y = [0.0], [x_0], [y_0], [v_x_0], [v_y_0]
    while sys.successful() and y[-1] >= 0:
        sys.integrate(sys.t + delta_t)
        t.append(sys.t)
        x.append(sys.y[0])
        y.append(sys.y[1])
        v_x.append(sys.y[2])
        v_y.append(sys.y[3])
    return 0.5*(x[-2] + x[-1])

In [ ]:
alphas = np.linspace(10.0, 80.0, 50)
ranges = [compute_range(alpha) for alpha in alphas]

In [ ]:
fig = figure(width=500, height=300,
             x_axis_label='alpha', y_axis_label='range')
fig.line(alphas, ranges)
show(fig)

## Maximal range

In [ ]:
scipy.optimize.minimize_scalar(
    lambda alpha: -compute_range(alpha),
    bracket=(10.0, 80.0), method='golden'
)

# Drag

The equations of motion of a projectile fired by a cannon is given by:
$$
  \frac{d^2 \vec{r}}{d t^2} = -g \vec{e}_y - \frac{B_2}{m} v e^{-\frac{y}{y_d}} \vec{v} 
$$
Realistic values for $g = 9.81 \frac{m}{s^2}$, $\frac{B_2}{m} = 4\cdot10^{-5} m^{-1}$, $y_d = 10^4 m$.

In [ ]:
def rhs(t, Y, g, b_2_m, y_d):
    x, y, v_x, v_y = Y
    v = np.sqrt(v_x**2 + v_y**2)
    drag_factor = b_2_m*v*np.exp(-y/y_d)
    return np.array([
        v_x,
        v_y,
        -drag_factor*v_x,
        -g - drag_factor*v_y,
    ])

## Jacobian

Computing the Jacobian is not really hard, but a little tedious.  It can easily be done using sympy.

In [ ]:
y, v_x, v_y, g, b_2_m, y_d = sympy.symbols('y v_x v_y g b_2_m y_d')

In [ ]:
rhs_v_x = -b_2_m*sympy.sqrt(v_x**2 + v_y**2)*sympy.exp(-y/y_d)*v_x

In [ ]:
rhs_v_y = -g -b_2_m*sympy.sqrt(v_x**2 + v_y**2)*sympy.exp(-y/y_d)*v_y

In [ ]:
def jac_element(expr, symbol):
    return sympy.lambdify(
        (y, v_x, v_y, g, b_2_m, y_d),
        sympy.diff(expr, symbol)
    )

In [ ]:
dv_x_dy = jac_element(rhs_v_x, y)

In [ ]:
dv_x_dv_x = jac_element(rhs_v_x, v_x)

In [ ]:
dv_x_dv_y = jac_element(rhs_v_x, v_y)

In [ ]:
dv_y_dy = jac_element(rhs_v_y, y)

In [ ]:
dv_y_dv_x = jac_element(rhs_v_y, v_x)

In [ ]:
dv_y_dv_y = jac_element(rhs_v_y, v_y)

In [ ]:
def jac(t, Y, g, b_2_m, y_d):
    x, y, v_x, v_y = Y
    return np.array([
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0],
        [0.0, dv_x_dy(y, v_x, v_y, g, b_2_m, y_d), dv_x_dv_x(y, v_x, v_y, g, b_2_m, y_d), dv_x_dv_y(y, v_x, v_y, g, b_2_m, y_d)],
        [0.0, dv_y_dy(y, v_x, v_y, g, b_2_m, y_d), dv_y_dv_x(y, v_x, v_y, g, b_2_m, y_d), dv_y_dv_y(y, v_x, v_y, g, b_2_m, y_d)],
    ])

## Solution

In [ ]:
g, b_2_m, y_d = 9.81, 4.0e-5, 1.0e4

In [ ]:
init_values = compute_init_values(40.0)
x_0, y_0, v_x_0, v_y_0 = init_values

Define the ODE system, setting initial conditions and parameters.

In [ ]:
sys = scipy.integrate.ode(rhs, jac) \
           .set_integrator('dopri5') \
           .set_initial_value(init_values, 0.0) \
           .set_f_params(g, b_2_m, y_d) \
           .set_jac_params(g, b_2_m, y_d)

In [ ]:
delta_t = 0.5e-4

In [ ]:
t, x, y, v_x, v_y = [0.0], [x_0], [y_0], [v_x_0], [v_y_0]

In [ ]:
while sys.successful() and y[-1] >= 0:
    sys.integrate(sys.t + delta_t)
    t.append(sys.t)
    x.append(sys.y[0])
    y.append(sys.y[1])
    v_x.append(sys.y[2])
    v_y.append(sys.y[3])

In [ ]:
fig = figure(width=500, height=300,
             x_axis_label='x', y_axis_label='y')
fig.line(x, y)
show(fig)

## Total energy

It is useful to check that the total energy decreases monotonically.  This is the sum of the kinetic and the potential energy, so
$$
    \frac{E}{m} = \frac{v_x^2 + v_y^2}{2} + g y
$$

In [ ]:
def total_energy(x, y, v_x, v_y, g):
    return 0.5*(np.array(v_x)**2 + np.array(v_y)**2) + g*np.array(y)

In [ ]:
energy = total_energy(x, y, v_x, v_y, g)

In [ ]:
fig = figure(width=500, height=300,
             x_axis_label='t', y_axis_label='E')
fig.line(t, energy)
show(fig)

## Work

The work is defined as
$$
    W = \int \vec{F} d\vec{s}
$$
where $\vec{F}$ is the total force acting on the projectile and $d\vec{s}$ is the displacement.  The integral is taken along the projectile's path.

In [ ]:
def work(x, y, v_x, v_y, g, b_2_m, y_d):
    x = np.array(x)
    y = np.array(y)
    delta_x = x[1:] - x[:-1]
    delta_y = y[1:] - y[:-1]
    Y = np.vstack((x, y, np.array(v_x), np.array(v_y)))
    values = rhs(0.0, Y, g, b_2_m, y_d)
    F_x = 0.5*(values[2, 1:] + values[2, :-1])
    F_y = 0.5*(values[3, 1:] + values[3, :-1])
    return np.sum(F_x*delta_x + F_y*delta_y)

In [ ]:
work(x, y, v_x, v_y, g, b_2_m, y_d)

In [ ]:
energy[0] - energy[-1]

The work is indeed (almost) exactly equal to the energy difference between firing and impact.

## Maximal range

Again, we can compute the maximal range of the artillery piece by maximizing the following helper function.

In [ ]:
def helper_func(alpha):
    return -compute_range(alpha, rhs=rhs, jac=jac,
                          rhs_params=(g, b_2_m, y_d),
                          jac_params=(g, b_2_m, y_d))

In [ ]:
scipy.optimize.minimize_scalar(
    lambda alpha: -compute_range(
                        alpha, rhs=rhs, jac=jac,
                        rhs_params=(g, b_2_m, y_d),
                        jac_params=(g, b_2_m, y_d)
                   ),
    bracket=(10.0, 80.0), method='golden'
)

Taking drag into account, the firing angle to get the maximal range increases by almost a degree, and the range is almost halved.